In [5]:
import pandas as pd
import cv2
import os
import base64
import re
# List of YouTube video URLs
video_urls = [
    "https://www.youtube.com/watch?v=JDwduqfuHjc"
]

# Create an empty list to store the data for each video
video_data = []


# function to convert image to byte string
def image_to_byte_string(image):
    _, buffer = cv2.imencode('.jpg', image)
    byte_string = base64.b64encode(buffer).decode()
    return byte_string

# function to convert video to image
def vid2frames(video_chunk_num):
    # path to input video
    part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
    part_frame_folder_path = re.search(r"(p\d+)", video_chunk_num).group()
    part_frame_path = os.path.join(part_fpath,part_frame_folder_path)
    main_video_path =f'{os.path.join(part_fpath, video_chunk_num)}'
    
    #chunk_parts_video_fir_path p1,p2,p3,p4,p5
    print(part_frame_path)
    os.makedirs(part_frame_path, exist_ok = True)
   


    video_path = main_video_path
    #main_video_path VIDEid_p1.mp4
    print(video_path)


    # capture video
    cap = cv2.VideoCapture(video_path)
    print(cap)
    # loop through frames and save as images
    frame_count = 1
    while cap.isOpened():
        ret, frame = cap.read()
        print("h1")
        frames = []
        if ret:
            frame_name= f'{part_frame_path}/{video_chunk_num}-frame{frame_count}.jpg'
            # save frame as image
            cv2.imwrite(frame_name, frame)
            print(f'{video_chunk_num}/frame{frame_count}.jpg')
            
            frame = cv2.imread(frame_name)
            byte_string = image_to_byte_string(frame)
            frames.append(byte_string)
            #os.remove(frame_name)











            frame_count += 1
        else:
            break

    # release video capture and destroy window
    #os.remove(main_video_path)
    cap.release()
    cv2.destroyAllWindows()

def chunker(video_urls_list):
    # Loop through each video URL
    for url in video_urls_list:
        # Get the video ID from the URL
        video_id = url.split("=")[-1]
        # Download the video using youtube-dl
        os.system(f"youtube-dl -f 'bestvideo[ext=mp4]' -o 'video.mp4' {url}")
        # Open the video using OpenCV2
        cap = cv2.VideoCapture("JDwduqfuHjc.mp4")
        # Get the total number of frames
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        # Get the frame rate
        frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
        # Split the video into 10-second chunks with two frames per second
        chunk_size = 10 * frame_rate
        for i in range(0, total_frames, chunk_size):
            # Set the start and end frames for the chunk
            start_frame = i
            end_frame = min(i + chunk_size, total_frames)
            # Get the chunk of frames
            frames = []
            for j in range(start_frame, end_frame):
                # Read the frame
                ret, frame = cap.read()
                # Keep only the first and middle frame
                if j == start_frame or j == (start_frame + end_frame) // 2:
                    frames.append(frame)
            # Save the chunk as an mp4 file
            chunk_filename = f"{video_id}-p{int(i/chunk_size)+1}.mp4"

                # path to input video
            part_fpath = f'{os.path.join(os.getcwd(),"temp_dir_parts")}'
            os.makedirs(part_fpath, exist_ok = True)
            main_video_path =f'{os.path.join(part_fpath, chunk_filename)}'


            cf = r"C:\Users\SkyLake\Documents\Python 2.0\Proj02b\temp_dir_parts"
            frame_rate_chunk = 2
            out = cv2.VideoWriter(f"{main_video_path}", cv2.VideoWriter_fourcc(*'mp4v'), frame_rate_chunk, (frames[0].shape[1], frames[0].shape[0]))
            for frame in frames:
                out.write(frame)
            out.release()
            # Add the video data to the list
            video_data.append({
                "video_id": video_id,
                "video_url": url,
                "video_parts": chunk_filename,
                "frame_rate": frame_rate_chunk,
            })
        # Release the OpenCV2 capture object
        cap.release()


chunker(video_urls)
# Create the final data frame
df = pd.DataFrame(video_data)
print(df)


      video_id                                    video_url  \
0  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
1  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
2  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
3  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
4  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
5  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   
6  JDwduqfuHjc  https://www.youtube.com/watch?v=JDwduqfuHjc   

          video_parts  frame_rate  
0  JDwduqfuHjc-p1.mp4           2  
1  JDwduqfuHjc-p2.mp4           2  
2  JDwduqfuHjc-p3.mp4           2  
3  JDwduqfuHjc-p4.mp4           2  
4  JDwduqfuHjc-p5.mp4           2  
5  JDwduqfuHjc-p6.mp4           2  
6  JDwduqfuHjc-p7.mp4           2  
